In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
from PSG_PDF import PDF, report_data

from pylab import rcParams
rcParams['figure.figsize'] = 12,4

In [26]:
day = '20231220'

kine_list = [i.replace('\\','/') for i in sorted(glob(f'processed file/{day}/kine/*csv'))]
force_list = [i.replace('\\','/') for i in sorted(glob(f'processed file/{day}/force/*csv'))]

kine_name = [f"{os.path.basename(i).split('.')[0]}" for i in kine_list]
force_name = [f"{os.path.basename(i).split('.')[0]}" for i in force_list]

In [27]:
kine_data = {}
for k, n in zip(kine_list, kine_name):
    kine_data[n] = k
    
force_data = {}
for f, n in zip(force_list, force_name):
    force_data[n] = f

In [28]:
ks_cols = {
    'PELVIS_ANGLUAR_VELOCITY_Z'        : ['PELVIS'   , 'r'],
    'TORSO_ANGLUAR_VELOCITY_Z'         : ['TORSO'    , 'g'],
    'LEAD_ELBOW_ANGULAR_VELOCITY_X'    : ['ELBOW'    , 'b'],
    'LEAD_SHOULDER_ANGULAR_VELOCITY_Z' : ['SHOULDER' , 'y'],
}

ang_cols = {
    'TORSO_PELVIS_ANGULAR_VELOCITY_Z' : 'Hip-Shoulder Separation',
    'LEAD_ELBOW_ANGLE_X'              : 'ELBOW FLEXION',
    'LEAD_SHOULDER_ANGLE_Z'           : 'SHOULDER EXTERNAL ROTATION',          
    'LEAD_SHOULDER_ANGLE_X'           : 'SHOULDER HORIZONTAL ABDUCTION',
    'LEAD_KNEE_ANGULAR_VELOCITY_X'    : 'LEAD LEG KNEE EXTENSION ANGULAR VELOCITY',
    'LEAD_SHOULDER_ANGLE_Y'           : 'SHOULDER ABDUCTION', 
    'LEAD_KNEE_ANGLE_X'               : 'LEAD LEG KNEE FLEXION',
    'TORSO_ANGLE_X'                   : 'TRUNK FORWARD TILT',
    'TORSO_ANGLE_Y'                   : 'TRUNK LATERAL TILT',
}

ap_cols = {
    'REAR_FORCE_Y' : ['Trail Leg' , 'b'],
    'LEAD_FORCE_Y' : ['Stride Leg', 'r'],
}

vt_cols = {
    'REAR_FORCE_Z' : ['Trail Leg' , 'b'],
    'LEAD_FORCE_Z' : ['Stride Leg', 'r']
}

In [29]:
name = 'cheonyunnyeong_0001_R_80_185_OH_130_S'
kine = kine_data[name]
force = force_data[name]

In [30]:
player, trial, side, weight, height, type, ballspeed, _= name.split('_')

In [31]:
k_df = pd.read_csv(kine)
f_df = pd.read_csv(force)

k_sr = 180
k_kh_time = k_df['kh_time'][0]
k_fp_time = k_df['fp_time'][0] - k_kh_time
k_mer_time = k_df['mer_time'][0] - k_kh_time
k_br_time = k_df['br_time'][0] - k_kh_time

f_sr = 1080
f_kh_time = f_df['kh_time'][0] 
f_fp_time = f_df['fp_time'][0] - f_kh_time
f_mer_time = f_df['mer_time'][0] - f_kh_time
f_br_time = f_df['br_time'][0] - f_kh_time

k_df = k_df.iloc[k_kh_time:int(k_br_time + k_kh_time + (k_sr * 0.2)),:].reset_index(drop=True)
f_df = f_df.iloc[f_kh_time:int(f_br_time + f_kh_time + (f_sr * 0.2)),:].reset_index(drop=True)

k_df.drop(['kh_time','fc_time','fp_time','mer_time','br_time','mir_time'], axis=1, inplace=True)
f_df.drop(['kh_time','fc_time','fp_time','mer_time','br_time','mir_time'], axis=1, inplace=True)

k_len = len(k_df)
k_time = np.arange(0,k_len/k_sr, 1/k_sr)

f_len = len(f_df)
f_time = np.arange(0,f_len/f_sr, 1/f_sr)

k_fc_br = k_br_time - k_fp_time
f_fc_br = f_br_time - f_fp_time

In [32]:
y   = report_data.grf(f_df, ap_cols, f_time, f_fp_time, f_mer_time, f_br_time, axis='ap')
z   = report_data.grf(f_df, vt_cols, f_time, f_fp_time, f_mer_time, f_br_time, axis='vt')
ang = report_data.one_angle(k_df, ang_cols, k_time, k_fp_time, k_mer_time, k_br_time)
ks  = report_data.kinematic_sequence(k_df, ks_cols, k_time, k_fp_time, k_mer_time, k_br_time)